In [10]:
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [4]:
X = []
y = []

arq = open("./features_LBP_janela.txt", "r")
for linha in arq:
    aux = linha.split("|")
    lbp_local = []
    for i in range(len(aux)-1):
        classe_local = []
        aux2 = aux[i].split(";")
        for j in aux2:
            classe_local.append(float(j))
        lbp_local.append(classe_local)
    X.append(lbp_local)
    y.append(aux[len(aux)-1].replace("\n", ""))
arq.close()

In [17]:
y_preds = []
svm_preds = []
best_model_svm, best_acc_svm = None, 0.0

k_fold = 5
# Split dos folds
kfold = KFold(n_splits=k_fold, shuffle=True)

# For para treinamento entre os folds
fold_no = 1
for train, test in kfold.split(X, y):
    print('------------------------------------------------------------------------')
    print(f'Treinamento para o fold {fold_no} ...')

    # Vetores para teste e treinamento
    x_train_svm = []
    y_train_svm = []

    x_test_svm = []
    y_test_svm = []

    # Fazendo o split dos dados para treinamento
    for i in train:
        for j in range(6):
            x_train_svm.append(X[i][j])
            y_train_svm.append(y[i])

    # Fazendo o split dos dados para teste
    for i in test:
        for j in range(6):
            x_test_svm.append(X[i][j])
            y_test_svm.append(y[i])


    y_pred = y_test_svm[:]
    for res in y_pred:
        y_preds.append(res)

    # SVM
    svm_model = SVC(C=100, kernel='poly', gamma='scale', probability=True)
    svm_model.fit(x_train_svm, y_train_svm)
    svm_predictions = svm_model.predict_proba(x_test_svm)

    svm_predictions = svm_model.predict(x_test_svm)
    acc_svm = accuracy_score(y_test_svm, svm_predictions)

    for i in svm_predictions:
        svm_preds.append(i)

    print(f'{acc_svm} para o fold {fold_no}')

    if acc_svm > best_acc_svm:
        best_acc_svm = acc_svm
        best_model_svm = svm_model

    fold_no += 1


------------------------------------------------------------------------
Treinamento para o fold 1 ...
0.6452991452991453 para o fold 1
------------------------------------------------------------------------
Treinamento para o fold 2 ...
0.6680911680911681 para o fold 2
------------------------------------------------------------------------
Treinamento para o fold 3 ...
0.6324786324786325 para o fold 3
------------------------------------------------------------------------
Treinamento para o fold 4 ...
0.6217948717948718 para o fold 4
------------------------------------------------------------------------
Treinamento para o fold 5 ...
0.6004273504273504 para o fold 5


In [18]:
matriz_confusao = confusion_matrix(y_preds, svm_preds)
relatorio_classificacao = classification_report(y_preds, svm_preds)

In [19]:
print(relatorio_classificacao)

              precision    recall  f1-score   support

           0       0.64      0.58      0.61       468
           1       0.80      0.80      0.80       468
          10       0.62      0.73      0.67       468
          11       0.41      0.58      0.48       468
          12       0.40      0.27      0.32       468
          13       0.69      0.67      0.68       468
          14       0.72      0.68      0.70       468
           2       0.64      0.62      0.63       468
           3       0.84      0.88      0.86       468
           4       0.54      0.40      0.46       468
           5       0.63      0.75      0.68       468
           6       0.52      0.65      0.58       468
           7       0.74      0.57      0.64       468
           8       0.66      0.71      0.68       468
           9       0.69      0.63      0.66       468

    accuracy                           0.63      7020
   macro avg       0.64      0.63      0.63      7020
weighted avg       0.64   